# Food product ranking

Goals:
    
Pull a subset of the nutrients and calculate a modified version of each food product's "Nutri Score". This code will only use nutrients available in https://en.wikipedia.org/wiki/Nutri-Score. Note that this datbase is missing "Salt" and "Fruits and Vegetables" by measurement, so instead a search is performed on ingredients.

In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sal

from sqlalchemy import text

In [2]:
nutrient_matrix_data_p = r"../data/"

nutrient_matrix_csv_p = nutrient_matrix_data_p + "nutrients_matrix.csv.gz"

#### Import the data cleaned in another notebook. Set the fdc_id to the index.

In [3]:
nutrients_matrix = pd.read_csv(nutrient_matrix_csv_p)
nutrients_matrix.set_index("fdc_id", inplace = True)
print(nutrients_matrix.shape)
nutrients_matrix.head()

(1590701, 103)


,1003,1004,1005,1008,1079,1082,1084,1087,1089,1092,...,1099,1196,1316,1233,1112,1111,1273,1236,1080,1068
fdc_id,,,,,,,,,,,,,,,,,,,,,
344604,0.81,0.41,4.07,24.0,0.8,0.0,0.0,13.0,0.00,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344605,0.81,0.41,4.07,24.0,0.8,0.0,0.0,16.0,0.00,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344606,23.21,2.68,0.00,0.0,0.0,0.0,0.0,0.0,1.29,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344607,23.21,2.68,0.00,0.0,0.0,0.0,0.0,0.0,1.29,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344608,18.75,15.18,0.00,0.0,0.0,0.0,0.0,18.0,0.96,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Convert Kjoules to Kcal for energy, then combine the energy columns

In [4]:
nutrients_matrix['1062'] = nutrients_matrix['1062']/4.184
nutrients_matrix['1008'] = nutrients_matrix['1008'] + nutrients_matrix['1062']
del nutrients_matrix['1062']

#### Get the list of nutrient names from nourish

In [5]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [6]:
nourish_user = "gmichael"

nourish_pswd = "567khcwx3s"

engine = sal.create_engine('postgresql+psycopg2://' + nourish_user + ':' + nourish_pswd + '@awesome-hw.sdsc.edu/nourish')
conn = engine.connect()

In [7]:
query_nutrients = text('''SELECT *
from "usda_2022_nutrient_master"''')

result = conn.execute(query_nutrients)

nutrient_names = [i for i in result]

nutrient_names[0:2]

[(2047, 'Energy (Atwater General Factors)', 'KCAL', Decimal('957'), '280.0'),
 (2048, 'Energy (Atwater Specific Factors)', 'KCAL', Decimal('958'), '290.0')]

In [8]:
nutrient_names_df = pd.DataFrame(nutrient_names)
nutrient_names_df

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958,290.0
2,1001,Solids,G,201,200.0
3,1002,Nitrogen,G,202,500.0
4,1003,Protein,G,203,600.0
...,...,...,...,...,...
469,2061,"Ergosta-7,22-dienol",MG,None,16211.0
470,2062,"Ergosta-5,7-dienol",MG,None,16211.0
471,2063,Verbascose,G,None,2450.0
472,2064,Oligosaccharides,MG,None,2250.0


In [9]:
nutrient_names_df[nutrient_names_df['name'].str.contains('Energ')]

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958,290.0
9,1008,Energy,KCAL,208,300.0
63,1062,Energy,kJ,268,400.0


#### Pull the fdc_idcs and their ingredients / food categories from nourish

In [10]:
query_ingredients = text('''SELECT "fdc_id", "ingredients", "branded_food_category"
from "usda_2022_branded_food_product"''')

result = conn.execute(query_ingredients)

ingredient_data = [i for i in result]

conn.close()

In [11]:
ingredient_df = pd.DataFrame(ingredient_data)
ingredient_df.set_index("fdc_id", inplace = True)
print(ingredient_df.shape)
ingredient_df

(1702125, 2)


,ingredients,branded_food_category
fdc_id,,
355336,"Granola (Whole Grain Rolled Oats, Brown Sugar,...",
355337,"Ingredients: Raw cane sugar #, cocoa butter #,...",
355338,"INGREDIENTS: POTATO FLOUR, CANOLA OIL, CORNSTA...",
355339,"INGREDIENTS: SUGAR, UNBLEACHED ENRICHED FLOUR ...",
355340,INGREDIENTS: UNBLEACHED ENRICHED FLOUR (WHEAT...,
...,...,...
355332,"Ingredients: MILK**, sugar, vegetable fats (pa...",
355333,INGREDIENTS: UNBLEACHED ENRICHED FLOUR (WHEAT ...,
355334,"Ingredients: Sweeteners (isomalt, aspartame, a...",


#### Rename the nutrient columns and merge with ingredients

In [12]:
rename_dict = dict(zip(nutrient_names_df["id"].astype(str), nutrient_names_df["name"]))

In [13]:
nutrients_matrix.rename(columns = rename_dict, inplace = True)
nutrients_matrix

,Protein,Total lipid (fat),"Carbohydrate, by difference",Energy,"Fiber, total dietary","Fiber, soluble","Fiber, insoluble","Calcium, Ca","Iron, Fe","Potassium, K",...,"Fluoride, F","Choline, from phosphotidyl choline","PUFA 18:2 n-6 c,c",Glutamine,Vitamin D3 (cholecalciferol),Vitamin D2 (ergocalciferol),SFA 22:0,"Sugars, intrinsic",Lignin,Beta-glucans
fdc_id,,,,,,,,,,,,,,,,,,,,,
344604,0.81,0.41,4.07,24.0,0.8,0.0,0.0,13.0,0.00,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344605,0.81,0.41,4.07,24.0,0.8,0.0,0.0,16.0,0.00,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344606,23.21,2.68,0.00,0.0,0.0,0.0,0.0,0.0,1.29,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344607,23.21,2.68,0.00,0.0,0.0,0.0,0.0,0.0,1.29,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
344608,18.75,15.18,0.00,0.0,0.0,0.0,0.0,18.0,0.96,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2340755,4.85,1.82,7.58,67.0,0.6,0.0,0.0,30.0,0.55,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2340756,4.85,1.82,7.58,67.0,0.6,0.0,0.0,30.0,0.55,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2340757,4.85,1.82,7.58,67.0,0.6,0.0,0.0,30.0,0.55,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
nutrients_matrix = nutrients_matrix.merge(ingredient_df, left_index = True, right_index = True, how = 'left')
print(nutrients_matrix.shape)
nutrients_matrix.head(3)

(1590701, 104)


,Protein,Total lipid (fat),"Carbohydrate, by difference",Energy,"Fiber, total dietary","Fiber, soluble","Fiber, insoluble","Calcium, Ca","Iron, Fe","Potassium, K",...,"PUFA 18:2 n-6 c,c",Glutamine,Vitamin D3 (cholecalciferol),Vitamin D2 (ergocalciferol),SFA 22:0,"Sugars, intrinsic",Lignin,Beta-glucans,ingredients,branded_food_category
fdc_id,,,,,,,,,,,,,,,,,,,,,
344604,0.81,0.41,4.07,24.0,0.8,0.0,0.0,13.0,0.00,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,...",
344605,0.81,0.41,4.07,24.0,0.8,0.0,0.0,16.0,0.00,179.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,...",
344606,23.21,2.68,0.00,0.0,0.0,0.0,0.0,0.0,1.29,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"White Turkey, Natural Flavoring",


### Grab a subset of nutrients and prepare for ranking

In [15]:
cols = ["branded_food_category", "ingredients", "Energy", "Sugars, Total", "Fatty acids, total saturated",
        "Sodium, Na", "Fiber, insoluble", "Fiber, soluble", "Fiber, total dietary", "Protein"]
nutrients_matrix = nutrients_matrix[cols]
nutrients_matrix.head()

,branded_food_category,ingredients,Energy,"Sugars, Total","Fatty acids, total saturated","Sodium, Na","Fiber, insoluble","Fiber, soluble","Fiber, total dietary",Protein
fdc_id,,,,,,,,,,
344604,,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,...",24.0,2.44,0.00,203.0,0.0,0.0,0.8,0.81
344605,,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,...",24.0,2.44,0.00,203.0,0.0,0.0,0.8,0.81
344606,,"White Turkey, Natural Flavoring",0.0,0.00,0.89,67.0,0.0,0.0,0.0,23.21
344607,,"Turkey Breast, Natural Flavoring",0.0,0.00,0.89,67.0,0.0,0.0,0.0,23.21
344608,,"Turkey, natural Flavoring.",0.0,0.00,4.46,103.0,0.0,0.0,0.0,18.75


In [16]:
#turn off SettingWithCopyWarning
pd.options.mode.chained_assignment = None

#Merge related columns for fibers
nutrients_matrix["Fibers"] = (nutrients_matrix["Fiber, insoluble"] + nutrients_matrix["Fiber, soluble"] + 
                                 nutrients_matrix["Fiber, total dietary"])

nutrients_matrix.drop(columns=["Fiber, insoluble", "Fiber, soluble", "Fiber, total dietary"], inplace = True)
nutrients_matrix

,branded_food_category,ingredients,Energy,"Sugars, Total","Fatty acids, total saturated","Sodium, Na",Protein,Fibers
fdc_id,,,,,,,,
344604,,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,...",24.0,2.44,0.00,203.0,0.81,0.8
344605,,"Tomatoes, Tomato Juice, Less Than 2% Of: Salt,...",24.0,2.44,0.00,203.0,0.81,0.8
344606,,"White Turkey, Natural Flavoring",0.0,0.00,0.89,67.0,23.21,0.0
344607,,"Turkey Breast, Natural Flavoring",0.0,0.00,0.89,67.0,23.21,0.0
344608,,"Turkey, natural Flavoring.",0.0,0.00,4.46,103.0,18.75,0.0
...,...,...,...,...,...,...,...,...
2340755,Chocolate,"FILTERED WATER, ORGAIN ORGANIC PROTEIN BLEND (...",67.0,2.73,0.30,48.0,4.85,0.6
2340756,Chocolate,"FILTERED WATER, ORGAIN ORGANIC PROTEIN BLEND (...",67.0,2.73,0.30,58.0,4.85,0.6
2340757,Chocolate,"FILTERED WATER, ORGAIN ORGANIC PROTEIN BLEND (...",67.0,2.73,0.30,58.0,4.85,0.6


### Calculate a modified Nutri Score 
Modified because there is no fruits or vegetables %. Tried to follow the scoring mechanisms laid out here https://en.wikipedia.org/wiki/Nutri-Score

In [17]:
beverage_categories = ['Alcohol', 'Alcoholic Beverages', 'Baby/Infant  Foods/Beverages', 
                       'Baby/Infant – Foods/Beverages', 'Beer', 'Breakfast Drinks', 'Coffee', 
                       'Coffee - Instant, Roast and Ground', 'Coffee/Tea/Substitutes', 'Drinks', 
                       'Drinks - Energy Drinks', 'Drinks - Juices, Drinks and Cordials', 'Drinks - Powdered', 
                       'Drinks - Soft Drinks', 'Drinks Flavoured - Ready to Drink', 'Energy, Protein & Muscle Recovery Drinks', 
                       'Food/Beverage/Tobacco Variety Packs', 'Frozen Fruit & Fruit Juice Concentrates', 
                       'Fruit & Vegetable Juice, Nectars & Fruit Drinks', 'Iced & Bottle Tea', 'Infant Formula', 'Liquid Water Enhancer',
                       'Milk', 'Milk Additives', 'Milk/Cream', 'Milk/Cream - Shelf Stable', 'Milk/Milk Substitutes', 
                       'Non Alcoholic Beverages  Not Ready to Drink', 'Non Alcoholic Beverages  Ready to Drink',
                       'Non Alcoholic Beverages – Not Ready to Drink', 'Non Alcoholic Beverages – Ready to Drink',
                       'Other Drinks', 'Plant Based Milk', 'Plant Based Water', 'Powdered Drinks', 'Ready To Drink', 
                       'Soda', 'Sport Drinks', 'Tea - Bags, Loose Leaf, Speciality', 'Tea Bags', 'Water']

In [23]:
def get_energy_points(df):
    if df["branded_food_category"] in beverage_categories:
        energy_points = df["Energy"]//7.2
        energy_points = min(energy_points, 10)
    else:
        energy_points = df["Energy"]//80
        energy_points = min(energy_points, 10)
    return energy_points

In [24]:
def get_sugar_points(df):
    if df["branded_food_category"] in beverage_categories:
        sugar_points = df["Sugars, Total"]//1.5
        sugar_points = min(sugar_points, 10)
    else:
        sugar_points = df["Sugars, Total"]//4.5
        sugar_points = min(sugar_points, 10)
    return sugar_points

In [25]:
%%time
nutrients_matrix.apply(get_energy_points, axis=1)

CPU times: user 13.1 s, sys: 206 ms, total: 13.3 s
Wall time: 13.3 s


fdc_id
344604     0.0
344605     0.0
344606     0.0
344607     0.0
344608     0.0
          ... 
2340755    0.0
2340756    0.0
2340757    0.0
2340758    0.0
2340759    3.0
Length: 1590701, dtype: float64

Scale the current nutrients column based on the scaling factors for the nutri score.

In [9]:
#unhealthy nutrients (max of 10)
food_nutrients_df['Energy_NutriScore'] = food_nutrients_df["Energy"]//80
food_nutrients_df['Energy_NutriScore'] = food_nutrients_df['Energy_NutriScore'].apply(lambda x: min(x, 10))

food_nutrients_df['SatFat_NutriScore'] =  food_nutrients_df["Fatty acids, total saturated"]//1
food_nutrients_df['SatFat_NutriScore'] = food_nutrients_df['SatFat_NutriScore'].apply(lambda x: min(x, 10))

food_nutrients_df['Sugars_NutriScore'] = food_nutrients_df["Sugars"]//4.5
food_nutrients_df['Sugars_NutriScore'] = food_nutrients_df['Sugars_NutriScore'].apply(lambda x: min(x, 10))

#healthy nutrients (max of 5)
food_nutrients_df['Fibers_NutriScore'] = food_nutrients_df["Fibers"]//0.7
food_nutrients_df['Fibers_NutriScore'] = food_nutrients_df['Fibers_NutriScore'].apply(lambda x: min(x, 5))

food_nutrients_df['Protein_NutriScore'] = food_nutrients_df["Protein"]//1.6
food_nutrients_df['Protein_NutriScore'] = food_nutrients_df['Protein_NutriScore'].apply(lambda x: min(x, 5))

Create a +1 flag for certain items that are not nutrients but can be found in the ingredients

In [10]:
food_nutrients_df['ingredients_english'] = food_nutrients_df['ingredients_english'].str.upper()
food_nutrients_df['has_salt'] = np.where(food_nutrients_df['ingredients_english'].str.contains('SALT'),1,0)
food_nutrients_df['has_fruit'] = np.where(food_nutrients_df['ingredients_english'].str.contains('FRUIT'),1,0)
food_nutrients_df['has_legume'] = np.where(food_nutrients_df['ingredients_english'].str.contains('LEGUME'),1,0)
food_nutrients_df['has_nuts'] = np.where(food_nutrients_df['ingredients_english'].str.contains('NUTS'),1,0)

Calculate a modified nutri score. Negative values are better, positive values are worse

In [11]:
food_nutrients_df["nutri_score"] = ((food_nutrients_df['Energy_NutriScore'] + food_nutrients_df['SatFat_NutriScore'] + 
                                   food_nutrients_df['Sugars_NutriScore'] + food_nutrients_df['has_salt']) - 
                                    (food_nutrients_df['Fibers_NutriScore'] + food_nutrients_df['has_fruit'] +
                                    food_nutrients_df['has_legume'] + food_nutrients_df['Protein_NutriScore'] +
                                    food_nutrients_df['has_nuts']) )

food_nutrients_df.drop(columns = ['Energy_NutriScore', 'SatFat_NutriScore', 'Sugars_NutriScore', 'Fibers_NutriScore', 
                                 'Protein_NutriScore', 'has_salt', 'has_fruit', 'has_legume', 'has_nuts'], inplace = True)

Rank the nutri scores with grades A-E

In [12]:
best_score = food_nutrients_df["nutri_score"].min()
worst_score = food_nutrients_df["nutri_score"].max()
score_options = []
while(best_score < worst_score+1):
    score_options.append(best_score)
    best_score += 1

In [13]:
#group_size = (len(score_options) - 1)//5
group_size = math.ceil(len(score_options)/5)
grouping_dict = {}
counter = 0
for score in score_options:
    #grouping_dict[score] = counter//group_size
    grouping_dict[score] = math.floor(counter/group_size)
    counter += 1

In [14]:
grades = ['A', 'B', 'C', 'D', 'E']

In [15]:
food_nutrients_df['nutri_score_grouping'] = food_nutrients_df['nutri_score'].apply(lambda x: grades[grouping_dict[x]])

In [16]:
#do some basic clean up
food_nutrients_df.drop_duplicates(subset = "gtin_upc", inplace = True)
food_nutrients_df.sort_values(by = "nutri_score", inplace = True)
food_nutrients_df.drop(columns = ["NDB_No", "gtin_upc"], inplace = True)
food_nutrients_df

,NDB_Number,long_name,manufacturer,ingredients_english,Energy,"Fatty acids, total saturated",Protein,Sugars,Fibers,nutri_score,nutri_score_grouping
11503,45127857,Kashi Golean Powders Green Tea 1.41oz,Kellogg Company US,"PEA PROTEIN, GROUND FLAX SEEDS, GROUND CHIA SE...",0.0,1.90,54.10,2.60,35.5,-11.0,A
11504,45127861,Kashi Golean Powders Green Tea 30oz,Kellogg Company US,"PEA PROTEIN, GROUND FLAX SEEDS, GROUND CHIA SE...",0.0,1.90,54.10,2.60,35.5,-11.0,A
11505,45127873,Kashi Golean Powders Vanilla 30oz,Kellogg Company US,"PEA PROTEIN, GROUND FLAX SEEDS, GROUND CHIA SE...",0.0,1.80,52.30,2.50,30.2,-11.0,A
11795,45130569,ORVILLE REDENBACHERS White Corn,Conagra Brands,WHOLE GRAIN POPPING CORN.,0.0,0.00,10.00,0.00,15.0,-10.0,A
48605,45368386,Quaker Masa Harina de Maiz Corn Tortilla Mix 7...,QTG,GROUND CORN TREATED WITH LIME.,0.0,0.57,9.15,1.12,7.5,-10.0,A
...,...,...,...,...,...,...,...,...,...,...,...
27525,45222291,TOFFEE FONDUE SAUCE,TALLGRASS,"CANE SUGAR, SALTED BUTTER, HOMOGENIZED WHOLE M...",500.0,21.43,0.00,50.00,0.0,27.0,E
15165,45155456,"FRANKFORD, WHITE CHOCOLATE","Frankford Candy, LLC","SUGAR, HYDROGENATED PALM KERNEL OIL, DAIRY PRO...",540.0,28.00,0.00,60.00,0.0,27.0,E
25233,45210376,SIXLETS CANDY,SWEET WORKS,"SUGAR, DEXTROSE, PARTIALLY HYDROGENATED PALM K...",500.0,21.43,0.00,71.43,0.0,27.0,E
45465,45351536,TOFFEE,GRIFF'S COFFEE,"DARK CHOCOLATE (CHOCOLATE LIQUOR, COCOA BUTTER...",533.0,16.67,0.00,46.67,0.0,27.0,E


In [17]:
food_nutrients_df.to_csv("food_nutrient_ranking.csv", index = False)